In [178]:
#load the base function
rm(list = ls())
setwd(rprojroot::find_rstudio_root_file())
source("base/Preprocess_data.R")

# The preprocess_data() function

This function formats the predictors (X) and target variable (y). <br>
It then splits them into training (X_train, y_train, q_train) and test set (X_test,y_test,q_test). 

- The variable (q) represents the average monthly flow in each catchment.
- The variable (X) represents the list of predictors.
- The variables (y) represents the cummulative volume of water in a selected period of time.

This three most basic parameters are:
   - **catchment_code**. Corresponds to the DGA gauge unique code. For example: "5410002",
   - **datetime_initialisation**. Corresponds the date of start of the forecast, until that date the data are considered.  For example:'2016-5-1',
   - **predictor_list**. It's a list with predictors with the form 'variable'_'function of aggregation'_'aggregation period in months'. The variables must be within the datasets. For example: c("pr_sum_-1months","tem_mean_2months","SOI_last_1months")

In [179]:
data_EDA1 = preprocess_data(
    catchment_code = "5410002",
    datetime_initialisation = '2016-5-1',
    predictor_list = list("pr_sum_-1months","tem_mean_2months")
)

data_EDA1  %>% names()

[1] "X_train"      "y_train"      "q_train"      "wy_train"     "raw_data"    
 [6] "time_horizon" "info"         "X_test"       "y_test"       "q_test"

## function's arguments (parameters)
This three most basic parameters are:
   - **catchment_code**. Corresponds to the DGA gauge unique code. For example: "5410002",
   - **datetime_initialisation**. Corresponds the date of start of the forecast, until that date the data are considered.  For example:'2016-5-1',
   - **predictor_list**. It's a list with predictors with the form 'variable'_'function of aggregation'_'aggregation period in months'. The variables must be within the datasets. For example: c("pr_sum_-1months","tem_mean_2months","SOI_last_1months")
   
<br>
Whereas the optional variables are:

- **horizon**. It corresponds to the forecast period in months. Typically starts in September (month_start = 9) and finishes in March (month_end=3). If the tag 'window_method' = "dynamic, then the period begins the month of the initialisation date after month_start. For example (default): horizon_mode(window_method = "dynamic", month_start = 9, month_end = 3)
- **data_location_paths**. It corresponds to a list of local paths to the datasets. The default can be obtain using 'get_default_datasets_path' function. For example (default): 
    get_default_datasets_path(meteo = "ens30avg",hydro = "ERA5Ens_SKGE").<br>
    It's equivalent to:
    list( <br>
    catchments_attributes_filename='data_input/attributes/attributes_49catchments_ChileCentral.csv', <br>
    flows_filename ='data_input/flows/flows_mm_monthly_49catchments_ChileCentral.csv', <br>
    meteo_filename ='data_input/meteo_variables/meteo_monthly_catchments_ChileCentral_ens30avg_1979_present.csv', <br>
    hydro_filename ='data_input/storage_variables/hydro_variables_monthly_catchments_ChileCentral_ERA5Ens_SKGE.csv', <br>
    climateindex_filename ='data_input/climate_index_variables/indices_mensuales_1979_present')
     <br>

- **water_units**. It corresponds to the destination/target units of the flow(q) and volume(y). For example (default): waterunits(q = "m^3/s", y = "GL")
- **forecast_mode**. It corresponds to the methods or mode in which the forecast will be apply afterwards. It only affects the  preprocess_data in the inclusion of the test set. For example: "both" (default),"cv","prediction"
- **remove_wys**. Remove select water years from the datasets so they are not included in the training. For example: c(1995,2019,1930). default: NULL.

In [180]:
# run with all the parameters specified
data_EDA2 = preprocess_data(
    catchment_code = "5410002",
    datetime_initialisation = '2016-5-1',
    predictor_list = list("pr_sum_-1months","tem_mean_2months"),
    horizon = horizon_mode(window_method = "dynamic", month_start = 9, month_end = 3),
    data_location_paths = get_default_datasets_path(meteo = "ens30avg",hydro = "ERA5Ens_SKGE"),
    water_units = waterunits(q = "m^3/s", y = "GL"),
    forecast_mode = "both",
    remove_wys <- NULL
)

data_EDA2  %>% names()

[1] "X_train"      "y_train"      "q_train"      "wy_train"     "raw_data"    
 [6] "time_horizon" "info"         "X_test"       "y_test"       "q_test"

In [181]:
# check if they have all the components equal
all.equal(data_EDA1, data_EDA2)

[1] TRUE

## function's outputs
- **X_train**: The predictor variables for the training set.<br>
- **y_train**: The target variable (volume) for the training set.<br>
- **q_train**: The flow data for the training set.<br>
- **wy_train**: The water year vector for the training set.<br>
- **raw_data**: The raw input data used to generate the training and test sets.<br>
- **time_horizon**: The forecast horizon used in the analysis.<br>
- **info**: A list containing the arguments of the function.<br>
- **X_test**: The predictor variables for the test set.(may be NULL or may not exists).<br>
- **y_test**: The target variable (volume) for the test set. (may be NULL or may not exists).<br>
- **q_test**: The flow data for the test set. (may be NULL or may not exists)



<br>
<br>
<br>
<br>
<br>
<br>

Let's have a look in the internal functions used in **preprocess_data** function

# Internal steps of the pre-processing

## get_default_datasets_path()
The function **get_default_datasets_path** returns a list of default file paths for the datasets required. 

It takes two arguments, **meteo** and **hydro**, which specify the type of meteorological and hydrological datasets to be used. By default, the function sets: meteo = "ens30avg" and hydro = "ERA5Ens_SKGE". **meteo and hydro can be set to NULL**.

The returned list contains five elements, each corresponding to a file path for a specific dataset. These include the catchment attributes, flow data, meteorological data, hydrological data, and climate index data. The paths are stored in the list under the following names:

- **catchments_attributes_filename**
- **flows_filename**
- **meteo_filename**
- **hydro_filename**
- **climateindex_filename**




In [182]:
data_location_paths <- get_default_datasets_path(meteo = "ens30avg",hydro = "ERA5Ens_SKGE")
data_location_paths

$catchments_attributes_filename
[1] "data_input/attributes/attributes_49catchments_ChileCentral.csv"

$flows_filename
[1] "data_input/flows/flows_mm_monthly_49catchments_ChileCentral.csv"

$meteo_filename
data_input/meteo_variables/meteo_monthly_catchments_ChileCentral_ens30avg_1979_present.csv

$hydro_filename
data_input/storage_variables/hydro_variables_monthly_catchments_ChileCentral_ERA5Ens_SKGE.csv

$climateindex_filename
[1] "data_input/climate_index_variables/indices_mensuales_1979_present.csv"

## read_catchment_data()
The function **read_catchment_data** reads data related to a particular catchment (a specific area where water is collected) and returns information on the water flows, meteorological, and other relevant variables.

The function takes as input the DGA unique **catchment_code**, which identifies the specific catchment to be analyzed, and other optional parameters like the removal of specific water years, unit conversion, and file paths for the data. 

The output of the function includes the monthly flows, meteorological and hydrological data, climate index data, the raw data dataframe, and attributes of the catchment data.

In [183]:
#PARAMETERS
catchment_code <- "5410002"

# catchment data (raw forcings, flows)
catchment_data <- read_catchment_data(
catchment_code = catchment_code,
remove_wys = c(1990,1950,2013),
water_units = waterunits(q = "m^3/s", y = "GL"),# GL is equivalent to one million m3,
data_location_paths = data_location_paths
)
#attributes of the catchment data
print(names(catchment_data))

[1] "monthly_flows"        "monthly_meteo"        "monthly_hydro"       
[4] "monthly_climateindex" "raw_data_df"          "attributes_catchment"


In [184]:
print('catchment attributes:')
catchment_data$attributes_catchment

[1] "catchment attributes:"


,cod_cuenca,gauge_name,gauge_lat,gauge_lon,area_km2
,<int>,<chr>,<dbl>,<dbl>,<dbl>
21,5410002,Rio Aconcagua En Chacabuquito,-32.8503,-70.5094,2113.423


The output represents a data frame that contains catchment attributes such as catchment code, gauge name, gauge latitude, gauge longitude, and catchment area in km2. In this case, the data frame only contains one row with values corresponding to catchment code "5410002", gauge name "Rio Aconcagua En Chacabuquito", gauge latitude "-32.8503", gauge longitude "-70.5094", and catchment area of "2113.423" square kilometers.

In [185]:
print('monthly averaged flows:')
catchment_data$monthly_flows  %>% tail()

[1] "monthly averaged flows:"


wy_simple,wym,Q_mm,Q_converted
<int>,<dbl>,<dbl>,<dbl>
2020,10,70.89504,56.97436
2020,11,42.97035,34.53285
2020,12,28.80140,23.14607
2021,1,18.07564,14.52637
2021,2,16.04798,12.89686
2021,3,17.81437,14.31641


The output shows the monthly averaged flows of the catchment. The column headers "wy_simple" represents water year, "wym" represents water year month, "Q_mm" represents monthly streamflow in mm, and "Q_converted" represents monthly streamflow in cubic meters per second (m³/s).

In [186]:
print('monthly averaged meteorology:')
catchment_data$monthly_meteo %>% tail()

[1] "monthly averaged meteorology:"


wy_simple,wym,pr,tem
<int>,<dbl>,<dbl>,<dbl>
2022,4,95.740,-1.161
2022,5,41.339,0.120
2022,6,1.090,0.557
2022,7,2.667,3.991
2022,8,1.985,8.589
2022,9,2.621,10.640


The output shows the monthly meteorological data of the catchment. The data includes four columns 'wy_simple' which stands for water year, 'wym' which stands for the water year month, 'pr' which stands for monthly accumulated precipitation (in mm), and 'tem' which stands for average monthly temperature (in Celsius).

In [187]:
print('monthly averaged hydrological model variables:')
catchment_data$monthly_hydro %>% tail()

[1] "monthly averaged hydrological model variables:"


wy_simple,wym,AE,SLZ,SM,SP,SUZ,STORAGE
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2019,7,0.442,41.749,165.863,0,0,208.054
2019,8,0.544,30.032,152.008,0,0,182.584
2019,9,0.565,21.430,136.439,0,0,158.434
2019,10,0.420,19.015,124.232,0,0,143.667
2019,11,0.349,13.823,112.935,0,0,127.107
2019,12,0.242,9.828,103.779,0,0,113.849


This output shows monthly averaged hydrological model variables for the specific catchment. The columns represent different hydrological model variables (this may vary depending on the chosen hydrological model). for this specific case the columns are Actual Evapotranspiration (AE), Surface Lateral Discharge (SLZ), Soil Moisture (SM), Surface Runoff (SP), Subsurface Upper Zone Flow (SUZ), and Total Storage (STORAGE) and the rows correspond to different months in a water year.

In [188]:
print('monthly selected climate indices:')
catchment_data$monthly_climateindex %>% tail()

[1] "monthly selected climate indices:"


,wy_simple,wym,MEIv2,PDO,SOI,ONI,OLR,NINO1.2,NINO3,NINO4,NINO3.4,ESPI,AAO,BIENSO
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
506,2022,5,-1.79,-1.945,1.7,-0.91,1.2,-0.60,-0.67,-1.09,-0.98,-1.23034,0.7313,-1.52
507,2022,6,-1.78,-1.724,2.7,-1.01,1.2,-1.02,-0.96,-1.17,-1.07,-1.16183,1.4685,-1.87
508,2022,7,-1.75,-1.346,2.8,-0.99,1.2,-1.79,-1.10,-1.12,-0.98,-1.19816,0.3303,-1.78
509,2022,8,-1.53,-1.986,0.5,-0.92,1.3,-1.13,-0.94,-0.99,-0.90,-0.79040,1.7134,-0.62
510,2022,9,-1.26,-1.837,3.5,-0.82,1.7,-0.46,-0.81,-0.84,-0.85,-1.23068,1.7004,-2.01
511,2022,10,-1.12,-0.960,2.3,-0.71,1.8,-0.57,-0.55,-0.66,-0.71,-0.98883,2.3037,-1.29


This output shows the monthly values of selected climate indices for a specific catchment, including the water year and month in which the data was estimated. The climate indices include MEIv2, PDO, SOI, ONI, OLR, NINO1.2, NINO3, NINO4, NINO3.4, ESPI, AAO, and BEST (BIENSO).

## initialisation_dates()

The function **initialisation_dates** takes a character string representing a date in the format "YYYY-MM-DD" and returns a list with several components related to the initialisation date.

In [189]:
datetime_initialisation = '2022-5-01'
date_init = initialisation_dates(datetime_initialisation)
date_init

$init_water_year_month
[1] 2

$init_water_year
[1] 2022

$init_month
[1] 5

$init_year
[1] 2022

$ymd_datetime
[1] "2022-05-01 UTC"

$datetime_initialisation
[1] "2022-5-01"


## predictors_generator()

The function **predictors_generator** creates a data frame of predictor variables for a catchment based on the specified list of predictors and initialisation date.

The function takes four arguments:

- **predictor_list**. It's a list with predictors with the form 'variable'_'function of aggregation'_'aggregation period in months'. The variables must be within the datasets. For example: c("pr_sum_-1months","tem_mean_2months","SOI_last_1months")

- **month_initialisation_index**: an integer value indicating the month of the water year in which the model is being initialised.
- **catchment_data**: a data frame containing the catchment data, including the target variable to be predicted.
- **remove_wys**. Remove select water years from the datasets so they are not included in the training. For example: c(1995,2019,1930). default: NULL.

The output is a data frame of predictor variables, with each row representing a water year, and columns corresponding to each of the predictor variables. The first column is the water year, and the remaining columns are the predictor variables specified in predictor_list.

In [190]:
#set specific predictors 'variable'_'agg-function'_'agg-months'
predictor_list <- c("pr_sum_-1months","tem_mean_5months","SOI_mean_1months")

# create the predictors variables
predictors <-
predictors_generator(
predictor_list = predictor_list,
month_initialisation_index = date_init$init_water_year_month,
catchment_data = catchment_data,
remove_wys = remove_wys
)

predictors  %>% tail()

Warning. Removing INCOMPLETE PREDICTORS, WATER-YEARS: c(1979, 1979)c(2023, 2023)



wy_simple,pr_sum_1months,tem_mean_5months,SOI_mean_1months
<dbl>,<dbl>,<dbl>,<dbl>
2017,15.941,9.3956,-0.3
2018,1.185,10.2272,0.8
2019,2.220,10.3064,0.2
2020,6.048,11.2954,0.3
2021,0.107,9.8326,0.6
2022,20.373,9.9900,2.8


## horizon_mode()

The **horizon_mode** function is used to define the forecast horizon. It takes three arguments:

- **window_method** a string indicating the method to compute the forecast window, either "dynamic" or "static".If the tag 'window_method' = "dynamic", then the period begins the month of the initialisation date after month_start. 
    If the tag 'window_method' = "static", the forecast target period remains static between month_start and month_end.

- **month_start** an integer indicating the starting month of the forecast horizon. In the example provided, it is set to September (month_start = 9)

- **month_end** an integer indicating the ending month of the forecast horizon. In the example provided, it is set to March (month_end = 3)

In [191]:
horizon = horizon_mode( 
    window_method = "dynamic",
    #forecast horizon in each year. Typically sep:9 to mar:3 for Chile.
    month_start = 9,
    month_end = 3)

# set target period of the forecast
forecast_horizon <- get_forecast_horizon(date_init,horizon)
forecast_horizon

$months_wy
 [1] "abr" "may" "jun" "jul" "ago" "sep" "oct" "nov" "dic" "ene" "feb" "mar"

$months_forecast_period
[1] "sep" "oct" "nov" "dic" "ene" "feb" "mar"

$dates_forecast_period
[1] "2022-09-01" "2022-10-01" "2022-11-01" "2022-12-01" "2023-01-01"
[6] "2023-02-01" "2023-03-01"

$days_per_month_horizon
Sep Oct Nov Dec Jan Feb Mar 
 30  31  30  31  31  28  31 

$months_before_initialisation
[1] "abr"

$init_forecast_index
[1] 6

$end_forecast_index
[1] 12

$month_initialisation_index
[1] 2

$date_initialisation
[1] "01 05 2022"

$datetime_initialisation
[1] "2022-5-01"

$volume_span_text
[1] "[sep,mar]"

$volume_span_text_v2
[1] "sep 2022 - mar 2023"

$months_target_water_year
 [1] "2022-04-01" "2022-05-01" "2022-06-01" "2022-07-01" "2022-08-01"
 [6] "2022-09-01" "2022-10-01" "2022-11-01" "2022-12-01" "2023-01-01"
[11] "2023-02-01" "2023-03-01"

## predictant_generator()

The function **predictant_generator** is used to create the target variable **predictant** which is a data frame containing two columns: volume and wy_simple.

- **forecast_horizon** is the output of the horizon_mode() function, which specifies the target period of the forecast.
- **catchment_data** is a data frame containing the catchment's data.  is the output of the read_catchment_data() function.
- **water_units** specifies the units in which the water volume is measured.

The output of predictant_generator is a data frame with the water volume (volume) for each water year (wy_simple).

In [192]:
# create the target variable (y:VOLUME)
predictant <- predictant_generator(
forecast_horizon = forecast_horizon,
catchment_data = catchment_data,
water_units = waterunits(q = "m^3/s", y = "GL")# GL is equivalent to one million m3,
)

predictant$y  %>% tail()

,volume,wy_simple
,<dbl>,<int>
33,722.7360,2015
34,767.5517,2016
35,402.6828,2017
36,337.1941,2018
37,237.9424,2019
38,770.8871,2020


## training_set()

The **training_set()** function separates predictor variables, predictant (target volume), and q (flow) into a training set removing the water_year_target. 

It returns a list containing four elements: X_train, y_train, q_train, and wy_train.

- **X_train** contains the training predictor variables, with n-years observations and p-variables. The variables are the ones asked using predictor_list. 
- **y_train** contains the training response variable, with n-years observations and 1 variable. The variable is identified as "volume", and represents the seasonal volume.
- **q_train** contains additional predictor variables that represent the volume of water in the months of horizon(). This data frame has n-years observations and m-months variables, each corresponding to a different month.
- **wy_train** It's a list of the training years (based on the available data in catchment_data)


In [193]:

train_set <- 
training_set(
predictor = predictors,
predictant = predictant,
water_year_target = date_init$init_water_year
)
train_set  %>% names

[1] "X_train"  "y_train"  "q_train"  "wy_train"

In [194]:
train_set

,pr_sum_1months,tem_mean_5months,SOI_mean_1months
,<dbl>,<dbl>,<dbl>
1981,2.253,8.9372,-0.2
1982,0.225,9.3870,0.1
1983,18.444,8.4892,-1.5
1984,4.312,9.5076,0.6
1985,4.425,8.6972,1.9
1986,23.608,9.4676,0.5
1987,14.411,9.6464,-2.3
1988,4.034,8.8672,0.2
1989,9.789,9.9268,2.7


## test_set()
The **testing_set** function takes in three arguments: predictors, predictant, and water_year_target.

- **predictors** represents a data frame or matrix containing the predictor variable(s).
- **predictant** represents a vector or data frame containing the response variable.
- **water_year_target** represents the target water year for the test set.

The output of the function is a list with three elements:
- **X_test** a data frame containing the predictor variables for the test year.
- **y_test** a data frame containing the predictant variables for the test year.
- **q_test** a data frame containing the monthly flow for the test year.

Output can be NULL if not data are available for the water_year_target


In [195]:
test_set  <- 
  testing_set(
    predictors = predictors,
    predictant = predictant,
    water_year_target = date_init$init_water_year)
test_set

,pr_sum_1months,tem_mean_5months,SOI_mean_1months
,<dbl>,<dbl>,<dbl>
2022,20.373,9.99,2.8


That's the end of the description of the different output, functions and steps to get the results from the function preprocess_data()